## Imports

In [ ]:
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, utils
from collections import Counter
from PIL import Image
from tqdm import tqdm
from pathlib import Path

import os
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models.detection as detection

from ultralytics import YOLO

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()  

[WARNING 2025-12-04 12:52:20,639 _showwarnmsg:109] /home/raclax/Documents/M2/Part2/DL2/Project/deepl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 


[WARNING 2025-12-04 12:52:20,639 _showwarnmsg:109] /home/raclax/Documents/M2/Part2/DL2/Project/deepl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 


pix2text is available


# YOLO v8

### model

In [ ]:
model = YOLO('yolov8s.pt')

results = model.train(
    data=root+'/data.yaml',
    epochs=10,
    imgsz=256,
    batch=16,
    name='math_symbols_detector'
)

results = model.val()

New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.234 🚀 Python-3.11.5 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 3769MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=YOLO_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=math_symbols_de

### Predictions and eval

In [ ]:
results = model.predict(root+'/images/train/001-equation002.png', save=True, conf=0.5)
result = results[0]

boxes = result.boxes
print("Boxes (xyxy):", boxes.xyxy)
print("Boxes (xywhn):", boxes.xywhn)
print("Confidence:", boxes.conf)
print("Classes:", boxes.cls)

for i, box in enumerate(boxes):
    xyxy = box.xyxy[0].tolist()
    conf = box.conf[0].item() 
    cls_id = int(box.cls[0].item()) 
    cls_name = result.names[cls_id] 
    
    print(f"Box {i}: {cls_name} (conf: {conf:.2f}) -> [{xyxy[0]:.1f}, {xyxy[1]:.1f}, {xyxy[2]:.1f}, {xyxy[3]:.1f}]")

In [ ]:
def evaluate_yolo_model_simple(model, data_yaml_path, imgsz=640, conf=0.25):
    metrics = model.val(
        data=data_yaml_path,
        imgsz=imgsz,
        conf=conf,
        split='val',
        plots=False 
    )
    

    results = {
        'mAP50': metrics.box.map50,       # mAP à IoU=0.5
        'mAP50-95': metrics.box.map,      # mAP moyenne sur 0.5:0.95
        'precision': metrics.box.p,       # Précision globale
        'recall': metrics.box.r,          # Rappel global
        'fitness': metrics.box.fitness    # Score de fitness
    }
    
    print('=== RÉSULTATS D\'ÉVALUATION SIMPLIFIÉE ===')
    print("mAP@0.5: ",results['mAP50'])
    print("mAP@0.5:0.95: ",results['mAP50-95'])
    print("Précision (P):",results['precision'])
    print("Rappel (R):",results['recall'])
    
    return results


data_config = root+'/data.yaml' 

metrics = evaluate_yolo_model_simple(model, data_config, conf=0.25)

Ultralytics 8.3.234 🚀 Python-3.11.5 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 3769MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 35.1±23.3 MB/s, size: 1.8 KB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 35.1±23.3 MB/s, size: 1.8 KB)
val: Scanning /home/raclax/Documents/M2/Part2/DL2/Project/DL_project_marie_clara/YOLO_dataset/labels/val.cache... 1999 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1999/1999 1.8Mit/s 0.0s0s
val: Scanning /home/raclax/Documents/M2/Part2/DL2/Project/DL_project_marie_clara/YOLO_dataset/labels/val.cache... 1999 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1999/1999 1.8Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 125/125 7.4it/s 16.9s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 125/125 7.4it/s 16.9s
                   all       1999      19394      0.696      0.695      